In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
import torchvision.models as models
import numpy as np
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
import numpy as np
from sklearn.preprocessing import label_binarize

# Set the device for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the labeled dataset
dataset = ImageFolder(root='C:/Users/sevmou2300/Desktop/Home/Postdoc/Task1_Plant_categorisation/Plantvation+Holmen-Spring23/Side camera/ResizedSelected', transform=transform)

# Split the dataset into training and test sets
train_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Create the pre-trained ResNet model and modify it as a feature extractor
num_classes = len(dataset.classes)
resnet = models.resnet18(pretrained=True)
# Freeze all layers in the ResNet model so they are not updated during training
for param in resnet.parameters():
    param.requires_grad = False
# Replace the last fully connected layer with a new one that has the correct number of output classes
resnet.fc = nn.Linear(resnet.fc.in_features, num_classes)
model = resnet.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

# Training loop
num_epochs = 10

# Create some empty arrays to store logs 
loss_log = []
accuracy_log = []

for epoch in range(num_epochs):
    # Training phase
    print(222)
    model.train()
    train_loss = 0.0
    train_correct = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        _, predictions = torch.max(outputs, 1)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_correct += torch.sum(predictions == labels).item()

    train_loss /= len(train_loader.dataset)
    train_accuracy = 100.0 * train_correct / len(train_loader.dataset)

    # Print training progress
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {train_loss:.4f} - Accuracy: {train_accuracy:.2f}%")
    
    # Store training stats after each epoch
    loss_log.append(train_loss)
    accuracy_log.append(train_accuracy)

# Testing phase
model.eval()
test_loss = 0.0
test_correct = 0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predictions = torch.max(outputs, 1)

        loss = criterion(outputs, labels)

        test_loss += loss.item()
        test_correct += torch.sum(predictions == labels).item()

test_loss /= len(test_loader.dataset)
test_accuracy = 100.0 * test_correct / len(test_loader.dataset)

print(f"Test Loss: {test_loss:.4f} - Test Accuracy: {test_accuracy:.2f}%")

# Save the trained model
torch.save(model.state_dict(), "TL_Resnet18_Side.pth")




C:\ProgramData\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\ProgramData\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


222
Epoch 1/10 - Loss: 0.0314 - Accuracy: 56.69%
222
Epoch 2/10 - Loss: 0.0239 - Accuracy: 68.94%
222
Epoch 3/10 - Loss: 0.0213 - Accuracy: 71.69%
222
Epoch 4/10 - Loss: 0.0193 - Accuracy: 74.10%
222
Epoch 5/10 - Loss: 0.0186 - Accuracy: 75.70%
222
Epoch 6/10 - Loss: 0.0179 - Accuracy: 76.61%
222
Epoch 7/10 - Loss: 0.0172 - Accuracy: 77.57%
222
Epoch 8/10 - Loss: 0.0170 - Accuracy: 77.83%
222
Epoch 9/10 - Loss: 0.0166 - Accuracy: 78.24%
222
Epoch 10/10 - Loss: 0.0165 - Accuracy: 77.69%
Test Loss: 0.0172 - Test Accuracy: 77.25%


In [2]:
# Testing phase
import numpy as np
model.eval()
test_loss = 0.0
test_correct = 0
test_total = 0

test_probabilities = []  # Store predicted probabilities instead of class labels
test_labels_list = []

test_predictions = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        probabilities = torch.softmax(outputs, dim=1)
        _, predictions = torch.max(outputs, 1)

        loss = criterion(outputs, labels)

        test_loss += loss.item()
        test_correct += torch.sum(predictions == labels).item()
        test_total += labels.size(0)

        test_probabilities.extend(probabilities.cpu().tolist())  # Store probabilities
        test_labels_list.extend(labels.cpu().tolist())
        test_predictions.extend(predictions.cpu().tolist())  # Store predictions

test_loss /= len(test_loader.dataset)
test_accuracy = 100.0 * test_correct / test_total

# Convert the test_probabilities and test_labels_list to numpy arrays
test_probabilities = np.array(test_probabilities)
test_labels_array = np.array(test_labels_list)

# Calculate testing metrics
test_precision_weighted = precision_score(test_labels_array, test_predictions, average='weighted')
test_recall_weighted = recall_score(test_labels_array, test_predictions, average='weighted')
test_f1_weighted = f1_score(test_labels_array, test_predictions, average='weighted')
test_auc_weighted = roc_auc_score(label_binarize(test_labels_array, classes=np.unique(test_labels_array)), test_probabilities, average='weighted')

test_precision_macro = precision_score(test_labels_array, test_predictions, average='macro')
test_recall_macro = recall_score(test_labels_array, test_predictions, average='macro')
test_f1_macro = f1_score(test_labels_array, test_predictions, average='macro')
test_auc_macro = roc_auc_score(label_binarize(test_labels_array, classes=np.unique(test_labels_array)), test_probabilities, average='macro')

print(f"Test Loss: {test_loss:.4f} - Test Accuracy: {test_accuracy:.2f}%")
print(f"Test Weighted Precision: {test_precision_weighted:.4f} - Test Weighted Recall: {test_recall_weighted:.4f} - Test Weighted F1 Score: {test_f1_weighted:.4f} - Test Weighted AUC: {test_auc_weighted:.4f}")
print(f"Test Macro Precision: {test_precision_macro:.4f} - Test Macro Recall: {test_recall_macro:.4f} - Test Macro F1 Score: {test_f1_macro:.4f} - Test Macro AUC: {test_auc_macro:.4f}")

# Save the trained model
torch.save(model.state_dict(), "trained_model_train_test_split.pth")

# Save the results to an Excel file
results_dict = {
    'Test Loss': [test_loss],
    'Test Accuracy': [test_accuracy],
    'Test Weighted Precision': [test_precision_weighted],
    'Test Weighted Recall': [test_recall_weighted],
    'Test Weighted F1 Score': [test_f1_weighted],
    'Test Weighted AUC': [test_auc_weighted],
    'Test Macro Precision': [test_precision_macro],
    'Test Macro Recall': [test_recall_macro],
    'Test Macro F1 Score': [test_f1_macro],
    'Test Macro AUC': [test_auc_macro],
}

results_df = pd.DataFrame(results_dict)
results_df.to_excel("TL_Resnet18_Side.xlsx", index=False)

Test Loss: 0.0172 - Test Accuracy: 77.25%
Test Weighted Precision: 0.7840 - Test Weighted Recall: 0.7725 - Test Weighted F1 Score: 0.7749 - Test Weighted AUC: 0.9382
Test Macro Precision: 0.7846 - Test Macro Recall: 0.7628 - Test Macro F1 Score: 0.7706 - Test Macro AUC: 0.9424


In [6]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
import numpy as np
from sklearn.preprocessing import label_binarize